In [1]:
from zeta_py.zeta_client.accounts.pricing import Pricing
from solana.rpc.async_api import AsyncClient
from zeta_py.client import Client
from zeta_py.types import Network, Asset
import anchorpy
from solders.pubkey import Pubkey
from zeta_py import pda
from solana.rpc.commitment import Confirmed

conn = AsyncClient("https://api.mainnet-beta.solana.com", commitment=Confirmed, blockhash_cache=True)

wallet = anchorpy.Wallet.local()  # get local filesystem keypair wallet
print(wallet.public_key)

4mYxEXiFwe3ePEmoj7CYCbiW7wGTojUQCmdPqGaoVaUh


In [2]:
print(f"Balance: {(await conn.get_balance(wallet.public_key)).value / 1e9} SOL")

Balance: 0.24393317 SOL


In [3]:
client = await Client.load(Network.MAINNET, conn, wallet, [Asset.SOL, Asset.APT])

INFO:zeta_py.accounts.Account:Loaded account: State
INFO:zeta_py.accounts.Account:Loaded account: Pricing
INFO:zeta_py.accounts.Account:Loaded account: Clock
INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccount


StreamError: Error in path (parsing)
stream read less than specified amount, expected 5, found 0

In [4]:
print(client.balance)
print(client.positions)
print(client.open_orders)

0.009917
{<Asset.SOL: 0>: Position(size=0.001, cost_of_trades=0.021262)}
{<Asset.SOL: 0>: [Order(order_id=184467459183839589766535671, client_id=0, open_order_address=Pubkey(
    ciwBQfLdM97vFp8GwYYLczPgYoLfdkTVD6dhHAeGoWj,
), open_order_slot=0, fee_tier=0, info=OrderInfo(price=10.0, size=0.001, price_lots=10000000, size_lots=1), side=<Side.BID: 0>, tif_offset=0)]}


In [3]:
await client.deposit(0.000001)

INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccountManager
INFO:zeta_py.client.Client:Deposit of $1e-06 USDC to margin account Hu2yXbPESXyamLdvDemeASCFd3rHnpN2xBDUJpnuG5UX submitted


Signature(
    kCs4xjrg4ogm94a7Xcpw7az5Dti8BsS3iPYXM1tZp3eTUZ9Vx8QGQ5QioZtWos1epLZQfrCNEFELAdoo1SkLw1j,
)

In [8]:
from solana.transaction import Transaction
from zeta_py import utils

from zeta_py.zeta_client.instructions import (
    deposit_v2,
    initialize_cross_margin_account,
    initialize_cross_margin_account_manager,
)

tx = Transaction()
tx.add(
    initialize_cross_margin_account_manager(
        {
            "cross_margin_account_manager": client._cross_margin_account_manager_address,
            "authority": client.provider.wallet.public_key,
            "payer": client.provider.wallet.public_key,
            "zeta_program": client.exchange.program_id,
        }
    )
)
# tx.add(
#     initialize_cross_margin_account(
#         {"subaccount_index": 0},
#         {
#             "cross_margin_account": client.margin_account.address,
#             "cross_margin_account_manager": client._cross_margin_account_manager_address,
#             "authority": client.provider.wallet.public_key,
#             "payer": client.provider.wallet.public_key,
#             "zeta_program": client.exchange.program_id,
#         },
#     )
# )
# tx.add(
#     deposit_v2(
#         {"amount": utils.convert_decimal_to_fixed_int(0.001)},
#         {
#             "margin_account": client.margin_account.address,
#             "vault": client._combined_vault_address,
#             "user_token_account": client._user_usdc_address,
#             "socialized_loss_account": client._combined_socialized_loss_address,
#             "authority": client.provider.wallet.public_key,
#             "state": client.exchange.state.address,
#             "pricing": client.exchange.pricing.address,
#         },
#     )
# )

# TODO: grab recent blockhash??
signed_tx = client.provider.wallet.sign_transaction(tx)
signed_tx.verify_signatures()

True